# Physics informed Neural Net

Here we implement a  example from the paper by [Raissi et al 2019](https://www.sciencedirect.com/science/article/pii/S0021999118307125?casa_token=qU5V_mqCUkYAAAAA:vCncJgWLW6Ac8OJbVOGSN9Gii4Bh6-i2woHLkOpDF6UKbl2JeUowNwtfNs_62kZdMaAx5O_FHg)

## Solving the Burgers equation

See Appendix 1 of paper
- $du/dt + u du/dx - (0.01*\pi) d^2u/dx^2 =0$,  $x \in [-1,1], t \in [0,1]$
- $u(0,x)=-sin(\pi x)$
- $u(t,-1)=0$ if $u>0$
- $u(t,1)=0$ if $u<0$

Note that the boundary conditions are not properly defined for this equation. The sign of u at the boundary is ignored and the diffusion term has no boundary condition. 
This probably works because the flow is away from the boundary at both sides, due to the initial condition.


__NOTE__

There are issues in the combination of Flux and Zygote for this application. See for example [here](https://discourse.julialang.org/t/using-flux-for-a-neural-net-solution-to-differential-equations/41996/4)  and [here](https://discourse.julialang.org/t/gradient-error-in-flux-model-inputs/53259/2) and [here](https://github.com/FluxML/Flux.jl/issues/1464)

The root cause seems to be the immutability of arrays, see [here](https://fluxml.ai/Zygote.jl/stable/limitations/)

In [0]:
## Install Flux if needed
#import Pkg
#Pkg.add("Flux")

In [0]:
# With Julia 1.7+, this will prompt if neccessary to install everything, including CUDA:
using Flux
using LinearAlgebra
using Statistics
using Plots

##  Dense NN for $u(t,x)$

In [0]:
# Define our model of u(t,x) as a dense network
n1=50
n2=50
model = Chain(Dense(2 => n1, tanh), BatchNorm(n1), Dense(n1 => n2,tanh),Dense(n2 => 1,tanh)) #1 since u maps to R

In [0]:
model([0.5 0.5]') #random after initialization, but before training

In [0]:
u(1,2)

In [0]:
# settings
n_train = 1000   #number of training points for interior
n_test  = 1000   #number of testing points for interior
n_init_train = 30   #number of training points for initial condition
n_init_test  = 30   #number of testing points for initial condition
n_bound_train = 30  #number of training points at either boundary
n_bound_test = 30  #number of testing points at either boundary
xmin=-1.0
xmax=1.0
tmin=0.0
tmax=1.0

# Interior
function random_points(xmin,xmax,tmin,tmax,n)
    temp = rand(Float32,2,n)
    temp[1,:].=xmin.+(xmax-xmin).*temp[1,:]
    temp[2,:].=tmin.+(tmax-tmin).*temp[2,:]
    return temp
end
#  select training and testing points for interior
xt_train = random_points(xmin,xmax,tmin,tmax,n_train)
yt_prior = model(xt_train)
xt_test  = random_points(xmin,xmax,tmin,tmax,n_test)
# initial condition
xt_init_train = random_points(xmin,xmax,tmin,tmin,n_init_train)
yt_init_prior = model(xt_init_train)
xt_init_test  = random_points(xmin,xmax,tmin,tmin,n_init_test)
# left boundary
xt_left_train = random_points(xmin,xmin,tmin,tmax,n_bound_train)
yt_left_prior = model(xt_left_train)
xt_left_test  = random_points(xmin,xmin,tmin,tmax,n_bound_test)
# right boundary
xt_right_train = random_points(xmax,xmax,tmin,tmax,n_bound_train)
yt_right_prior = model(xt_right_train)
xt_right_test  = random_points(xmax,xmax,tmin,tmax,n_bound_test)

nothing


In [0]:

p_prior = scatter(xt_train[1,:], xt_train[2,:], zcolor=yt_prior[1,:], title="UNtrained network", legend=true,clims=(-1,1),label="u(x,t)",xlabel="x",ylabel="t")
scatter!(p_prior, xt_left_train[1,:], xt_left_train[2,:], zcolor=yt_left_prior[1,:],label="u(-1,t)")
scatter!(p_prior, xt_right_train[1,:], xt_right_train[2,:], zcolor=yt_right_prior[1,:],label="u(1,t)")
scatter!(p_prior, xt_init_train[1,:], xt_init_train[2,:], zcolor=yt_init_prior[1,:],label="u(x,0)")
plot(p_prior, layout=(1,1), size=(700,500))

In [0]:
n1=50
n2=50
model = Chain(Dense(2 => n1, tanh), Dense(n1 => 1,tanh)) #1 since u maps to R

u(x,t)    = model(reshape([x,t],2,1))[1]
du(x,t)   = gradient(u,x,t)
dudx(x,t) = du(x,t)[1]
dudt(x,t) = du(x,t)[2]

res(x,t)  = dudt(x,t) + u(x,t) * dudx(x,t) # Equation for the interior

res_init(x,t) = u(x,t)+sin(π*x)

res_left(x,t)  = u(x,t)
res_right(x,t) = u(x,t)

function interior_loss(xt)
    loss=0.0
    for ipoint=1:size(xt,2)
        x         = xt[1,ipoint]
        t         = xt[2,ipoint]
        loss  = loss + res(x,t)^2
    end
    return loss
end

function initial_loss(xt)
    loss=0.0
    for ipoint=1:size(xt,2)
        x         = xt[1,ipoint]
        t         = xt[2,ipoint]
        loss  = loss + res_init(x,t)^2
    end
    return loss
end

interior_loss(xt_train)+initial_loss(xt_init_train)

#opt = Flux.Adam(0.01)      # will store optimiser momentum, etc.
#pars = Flux.params(model)  # contains references to arrays in model
#gs = gradient(pars) do
#    return initial_loss(xt_init_train)
#end
#Flux.Optimise.update!(opt,pars,gs)

xt=xt_train[:,1:5]
pars = Flux.params(model)
grad = gradient(()->interior_loss(xt),pars)

In [0]:
interior_loss(xt_train)+initial_loss(xt_init_train)

opt = Flux.Adam(0.01)      # will store optimiser momentum, etc.
pars = Flux.params(model)  # contains references to arrays in model
gs = gradient(pars) do
    return interior_loss(xt_train)+initial_loss(xt_init_train)
end
Flux.Optimise.update!(opt,pars,gs)

In [0]:
pars = Flux.params(model)  # contains references to arrays in model
opt = Flux.Adam(0.01)      # will store optimiser momentum, etc.
#data = Flux.DataLoader((rand(2,10), rand(1,10)), batchsize=10, shuffle=false) #fake data

loss(x,y)=interior_loss(xt_train)+initial_loss(xt_init_train) #ignore data TODO: this is a mess

#Flux.train!(loss, pars, data, opt)

In [0]:
interior_loss(xt_train)+initial_loss(xt_init_train)

In [0]:
pars

In [0]:
model([0.25 0.75 0.25 0.75 ; 0.25 0.25 0.75 0.75]) #apply the trained model, with one point in each colunm

In [0]:
model # show summary

In [0]:
π